In [1]:
!!git clone https://github.com/zykls/folktables.git


["Cloning into 'folktables'...",
 'remote: Enumerating objects: 423, done.\x1b[K',
 'remote: Counting objects:   0% (1/173)\x1b[K',
 'remote: Counting objects:   1% (2/173)\x1b[K',
 'remote: Counting objects:   2% (4/173)\x1b[K',
 'remote: Counting objects:   3% (6/173)\x1b[K',
 'remote: Counting objects:   4% (7/173)\x1b[K',
 'remote: Counting objects:   5% (9/173)\x1b[K',
 'remote: Counting objects:   6% (11/173)\x1b[K',
 'remote: Counting objects:   7% (13/173)\x1b[K',
 'remote: Counting objects:   8% (14/173)\x1b[K',
 'remote: Counting objects:   9% (16/173)\x1b[K',
 'remote: Counting objects:  10% (18/173)\x1b[K',
 'remote: Counting objects:  11% (20/173)\x1b[K',
 'remote: Counting objects:  12% (21/173)\x1b[K',
 'remote: Counting objects:  13% (23/173)\x1b[K',
 'remote: Counting objects:  14% (25/173)\x1b[K',
 'remote: Counting objects:  15% (26/173)\x1b[K',
 'remote: Counting objects:  16% (28/173)\x1b[K',
 'remote: Counting objects:  17% (30/173)\x1b[K',
 'remote: Counting obje

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd


In [7]:
df = pd.read_csv('folktables/adult_reconstruction.csv')
df.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,gender,native-country,income,occupation
0,20,40,0,0,Private,Bachelors,13,Married-civ-spouse,Wife,White,Female,United-States,49100,Tech-support
1,40,21,0,0,Private,Some-college,10,Divorced,Own-child,White,Male,United-States,11500,Craft-repair
2,10,17,0,0,Private,11th,7,Never-married,Own-child,White,Male,United-States,2600,Other-service
3,50,51,0,0,Private,HS-grad,9,Married-civ-spouse,Husband,Asian-Pac-Islander,Male,Cambodia,38997,Sales
4,38,28,0,0,Private,Bachelors,13,Never-married,Not-in-family,White,Male,?,41400,Exec-managerial


Data Preprocessing

In [8]:
# drop missing values
df.replace('?', np.nan, inplace=True)
df = df.dropna()
print(df.info())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 45849 entries, 0 to 49530
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   hours-per-week  45849 non-null  int64 
 1   age             45849 non-null  int64 
 2   capital-gain    45849 non-null  int64 
 3   capital-loss    45849 non-null  int64 
 4   workclass       45849 non-null  object
 5   education       45849 non-null  object
 6   education-num   45849 non-null  int64 
 7   marital-status  45849 non-null  object
 8   relationship    45849 non-null  object
 9   race            45849 non-null  object
 10  gender          45849 non-null  object
 11  native-country  45849 non-null  object
 12  income          45849 non-null  int64 
 13  occupation      45849 non-null  object
dtypes: int64(6), object(8)
memory usage: 5.2+ MB
None
hours-per-week    0
age               0
capital-gain      0
capital-loss      0
workclass         0
education         0
education-num    

In [9]:
# mapping categorical features in subgroups

mapping = {
    'Private': 'Private',
    'Self-emp-not-inc': 'Private',
    'Self-emp-inc': 'Private',
    'Federal-gov': 'Government',
    'Local-gov': 'Government',
    'State-gov': 'Government',
    'Without-pay': 'Unemployed',
    'Never-worked': 'Unemployed',
    '?': 'Unknown'
}
education_mapping = {
    'Doctorate': 'Graduate',
    'Masters': 'Graduate',
    'Prof-school': 'Graduate',
    'Bachelors': 'College',
    'Some-college': 'College',
    'Assoc-acdm': 'College',
    'Assoc-voc': 'College',
    'HS-grad': 'High School',
    '12th': 'High School',
    '11th': 'Less than High School',
    '10th': 'Less than High School',
    '9th': 'Less than High School',
    '7th-8th': 'Less than High School',
    '5th-6th': 'Less than High School',
    '1st-4th': 'Less than High School',
    'Preschool': 'Less than High School'
}
marital_mapping = {
    'Married-civ-spouse': 'Married',
    'Married-spouse-absent': 'Married',
    'Married-AF-spouse': 'Married',
    'Never-married': 'Never Married',
    'Divorced': 'Separated/Divorced',
    'Separated': 'Separated/Divorced',
    'Widowed': 'Widowed'
}
continent_mapping = {
    # North America
    'United-States': 'North America',
    'Puerto-Rico': 'North America',
    'Canada': 'North America',
    'Outlying-US(Guam-USVI-etc)': 'North America',
    'Mexico': 'North America',

    # Central & South America
    'Honduras': 'Central & South America',
    'Jamaica': 'Central & South America',
    'Dominican-Republic': 'Central & South America',
    'Ecuador': 'Central & South America',
    'Columbia': 'Central & South America',
    'Guatemala': 'Central & South America',
    'Nicaragua': 'Central & South America',
    'El-Salvador': 'Central & South America',
    'Trinadad&Tobago': 'Central & South America',
    'Peru': 'Central & South America',
    'Haiti': 'Central & South America',
    'Cuba': 'Central & South America',

    # Asia
    'Laos': 'Asia',
    'Cambodia': 'Asia',
    'India': 'Asia',
    'Japan': 'Asia',
    'China': 'Asia',
    'Iran': 'Asia',
    'Philippines': 'Asia',
    'Vietnam': 'Asia',
    'Taiwan': 'Asia',
    'Thailand': 'Asia',
    'Hong': 'Asia',
    'Laos': 'Asia',

    # Europe
    'England': 'Europe',
    'Germany': 'Europe',
    'Greece': 'Europe',
    'Italy': 'Europe',
    'Poland': 'Europe',
    'Portugal': 'Europe',
    'Ireland': 'Europe',
    'France': 'Europe',
    'Hungary': 'Europe',
    'Scotland': 'Europe',
    'Yugoslavia': 'Europe',
    'Holand-Netherlands': 'Europe',

    # Unknown/Other
    '?': 'Unknown',
    'South': 'Unknown'  # unclear value
}

relationship_mapping = {
    'Wife': 'Married',
    'Husband': 'Married',
    'Own-child': 'Child',
    'Other-relative': 'Other-relative',
    'Not-in-family': 'Not-in-family',
    'Unmarried': 'Unmarried'
}

# Workclass privilege
privilege_workclass = {
    'Private': 'Not Privileged Workclass',
    'Government': 'Privileged Workclass',
    'Unemployed': 'Not Privileged Workclass',
    'Unknown': 'Unknown Workclass'
}

# Education privilege
privilege_education = {
    'Graduate': 'Privileged Education',
    'College': 'Privileged Education',
    'High School': 'Not Privileged Education',
    'Less than High School': 'Not Privileged Education'
}

# Marital privilege
privilege_marital = {
    'Married': 'Privileged Marital',
    'Never Married': 'Not Privileged Marital',
    'Separated/Divorced': 'Not Privileged Marital',
    'Widowed': 'Not Privileged Marital'
}

# Relationship privilege
privilege_relationship = {
    'Married': 'Privileged Relationship',
    'Child': 'Not Privileged Relationship',
    'Not-in-family': 'Not Privileged Relationship',
    'Unmarried': 'Not Privileged Relationship',
    'Other-relative': 'Not Privileged Relationship'
}

# Gender privilege
privilege_gender = {
    'Male': 'Privileged Gender',
    'Female': 'Not Privileged Gender'
}

# Race privilege
privilege_race = {
    'White': 'Privileged Race'
    # All others will be filled as Not Privileged Race
}

# Country
privilege_country = {
    # Privileged (mostly Western nations)
    'United-States': 'Privileged Country',
    'Canada': 'Privileged Country',
    'England': 'Privileged Country',
    'Germany': 'Privileged Country',
    'France': 'Privileged Country',
    'Ireland': 'Privileged Country',
    'Scotland': 'Privileged Country',
    'Holand-Netherlands': 'Privileged Country',
    'Greece': 'Privileged Country',
    'Italy': 'Privileged Country',
    'Poland': 'Privileged Country',
    'Portugal': 'Privileged Country',
    'Hungary': 'Privileged Country',

    # Not Privileged (Global South and developing regions)
    'Mexico': 'Not Privileged Country',
    'Puerto-Rico': 'Not Privileged Country',
    'Outlying-US(Guam-USVI-etc)': 'Not Privileged Country',
    'Honduras': 'Not Privileged Country',
    'Jamaica': 'Not Privileged Country',
    'Dominican-Republic': 'Not Privileged Country',
    'Ecuador': 'Not Privileged Country',
    'Columbia': 'Not Privileged Country',
    'Guatemala': 'Not Privileged Country',
    'Nicaragua': 'Not Privileged Country',
    'El-Salvador': 'Not Privileged Country',
    'Trinadad&Tobago': 'Not Privileged Country',
    'Peru': 'Not Privileged Country',
    'Haiti': 'Not Privileged Country',
    'Cuba': 'Not Privileged Country',
    'Laos': 'Not Privileged Country',
    'Cambodia': 'Not Privileged Country',
    'India': 'Not Privileged Country',
    'Japan': 'Not Privileged Country',
    'China': 'Not Privileged Country',
    'Iran': 'Not Privileged Country',
    'Philippines': 'Not Privileged Country',
    'Vietnam': 'Not Privileged Country',
    'Taiwan': 'Not Privileged Country',
    'Thailand': 'Not Privileged Country',
    'Hong': 'Not Privileged Country',
    'Yugoslavia': 'Not Privileged Country',
    'South': 'Not Privileged Country'
}


In [10]:
df['workclass_mapped'] = df['workclass'].map(mapping)
df['education_mapped'] = df['education'].map(education_mapping)
df['marital_mapped'] = df['marital-status'].map(marital_mapping)
df['continent'] = df['native-country'].map(continent_mapping)
df['relationship_mapped'] = df['relationship'].map(relationship_mapping)
df['race_privilige'] = df['race'].map(privilege_race).fillna('Not Privileged Race')
df['workclass_privilige'] = df['workclass_mapped'].map(privilege_workclass)
df['education_privilige'] = df['education_mapped'].map(privilege_education)
df['marital_privilige'] = df['marital_mapped'].map(privilege_marital)
df['relationship_privilige'] = df['relationship_mapped'].map(privilege_relationship)
df['country_privilege'] = df['native-country'].map(privilege_country).fillna('Not Privileged Country')


Statistical Parity in Regression

In [28]:
cat_features = ['workclass', 'education', 'marital-status', 'relationship', 'race', 'gender', 'native-country', 'occupation', 'workclass_mapped','education_mapped', 	'marital_mapped', 	'continent', 	'relationship_mapped', 	'race_privilige', 	'workclass_privilige', 	'education_privilige', 	'marital_privilige', 	'relationship_privilige', 	'country_privilege']

encoded_labels = {}

encoded_df = df.copy()  # Use .copy() to avoid the warning
label_encoder = LabelEncoder()

for i in cat_features:
  encoded_df.loc[:, i] = label_encoder.fit_transform(encoded_df[i])
  encoded_labels[i] = label_encoder.classes_

features = list(df.columns)
features.remove('income')

train_set, test_set = train_test_split(encoded_df, test_size=0.2)

X_train = np.array(train_set[features])
y_train = np.array(train_set['income'])

X_test = np.array(test_set[features])
y_test = np.array(test_set['income'])


print('Size train set:', train_set.shape[0])
print('Size test set:',test_set.shape[0])

Size train set: 36679
Size test set: 9170


In [20]:
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)

In [21]:
test_df = test_set.copy()
test_df['pred_income'] = y_pred
test_df.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,...,marital_mapped,continent,relationship_mapped,race_privilige,workclass_privilige,education_privilige,marital_privilige,relationship_privilige,country_privilege,pred_income
13212,40,40,0,0,3,11,9,2,0,4,...,0,3,1,1,0,0,1,1,1,39919.150817
44744,40,33,0,0,2,10,16,2,0,1,...,0,0,1,0,0,1,1,1,0,63614.150596
9110,20,18,0,0,2,15,10,4,3,4,...,1,3,0,1,0,1,0,0,1,6562.237174
24852,40,57,6849,0,1,15,10,6,1,4,...,3,3,2,1,1,1,0,0,1,35363.851187
21077,25,19,0,0,5,15,10,4,3,4,...,1,3,0,1,1,1,0,0,1,6846.232870


In [72]:
def get_sp_reg_matrix(data, attribute):

  features = data[attribute].unique()

  matrix = pd.DataFrame(0.0, index=features, columns=features)

  for i in features:
    for j in features:
      matrix.loc[i, j] = data[data[attribute]==i]['pred_income'].mean() - data[data[attribute]==j]['pred_income'].mean()

  return matrix

race_sp = get_sp_reg_matrix(test_df, 'race')
gender_sp = get_sp_reg_matrix(test_df, 'gender')
native_country_sp = get_sp_reg_matrix(test_df, 'native-country')
marital_status_sp = get_sp_reg_matrix(test_df,'marital-status')


In [78]:
decoded_labels = {}

for i in encoded_labels.keys():
  label_decoder = {}
  for j in range(len(encoded_labels[i])):
    label_decoder[encoded_labels[i][j]] = j
  decoded_labels[i] = label_decoder

In [79]:
def rename_matrix(matrix, faeature):

  for i in decoded_labels[faeature]:
    matrix = matrix.rename(columns={decoded_labels[faeature][i]:i}, index={decoded_labels[faeature][i]:i})

  return matrix

In [80]:
gender_sp = rename_matrix(gender_sp, 'gender')
race_sp = rename_matrix(race_sp, 'race')
native_country_sp = rename_matrix(native_country_sp, 'native-country')

In [81]:
gender_sp

,Male,Female
Male,0.000000,15154.374933
Female,-15154.374933,0.000000


In [82]:
race_sp

,White,Asian-Pac-Islander,Black,Amer-Indian-Eskimo,Other
White,0.000000,-567.950464,10383.206548,9809.783630,9269.278814
Asian-Pac-Islander,567.950464,0.000000,10951.157012,10377.734095,9837.229279
Black,-10383.206548,-10951.157012,0.000000,-573.422918,-1113.927733
Amer-Indian-Eskimo,-9809.783630,-10377.734095,573.422918,0.000000,-540.504816
Other,-9269.278814,-9837.229279,1113.927733,540.504816,0.000000


In [83]:
native_country_sp

,United-States,Japan,Mexico,Hungary,Thailand,Germany,Philippines,India,Cambodia,Taiwan,...,Yugoslavia,Hong,Scotland,Ecuador,Portugal,South,Greece,Honduras,France,Ireland
United-States,0.000000,-3675.555417,15552.443943,-13490.430048,16246.688938,-3406.355705,-1178.899339,-9592.084300,7051.227355,-7828.314414,...,-4686.211403,-5954.250961,-5872.483530,14821.726618,10912.813077,1624.860571,-11961.877294,18987.364033,1106.804032,33.615835
Japan,3675.555417,0.000000,19227.999360,-9814.874631,19922.244355,269.199713,2496.656078,-5916.528883,10726.782773,-4152.758996,...,-1010.655986,-2278.695544,-2196.928112,18497.282035,14588.368494,5300.415988,-8286.321876,22662.919451,4782.359449,3709.171252
Mexico,-15552.443943,-19227.999360,0.000000,-29042.873991,694.244995,-18958.799648,-16731.343282,-25144.528243,-8501.216588,-23380.758357,...,-20238.655346,-21506.694904,-21424.927473,-730.717325,-4639.630866,-13927.583372,-27514.321236,3434.920091,-14445.639911,-15518.828108
Hungary,13490.430048,9814.874631,29042.873991,0.000000,29737.118986,10084.074344,12311.530709,3898.345748,20541.657404,5662.115635,...,8804.218645,7536.179087,7617.946519,28312.156666,24403.243125,15115.290620,1528.552755,32477.794082,14597.234080,13524.045883
Thailand,-16246.688938,-19922.244355,-694.244995,-29737.118986,0.000000,-19653.044643,-17425.588277,-25838.773238,-9195.461583,-24075.003352,...,-20932.900341,-22200.939899,-22119.172468,-1424.962320,-5333.875861,-14621.828367,-28208.566232,2740.675095,-15139.884906,-16213.073103
Germany,3406.355705,-269.199713,18958.799648,-10084.074344,19653.044643,0.000000,2227.456366,-6185.728596,10457.583060,-4421.958709,...,-1279.855698,-2547.895257,-2466.127825,18228.082322,14319.168782,5031.216276,-8555.521589,22393.719738,4513.159737,3439.971539
Philippines,1178.899339,-2496.656078,16731.343282,-12311.530709,17425.588277,-2227.456366,0.000000,-8413.184961,8230.126694,-6649.415075,...,-3507.312064,-4775.351622,-4693.584191,16000.625957,12091.712416,2803.759910,-10782.977955,20166.263372,2285.703371,1212.515174
India,9592.084300,5916.528883,25144.528243,-3898.345748,25838.773238,6185.728596,8413.184961,0.000000,16643.311656,1763.769887,...,4905.872897,3637.833339,3719.600771,24413.810918,20504.897377,11216.944871,-2369.792993,28579.448334,10698.888332,9625.700135
Cambodia,-7051.227355,-10726.782773,8501.216588,-20541.657404,9195.461583,-10457.583060,-8230.126694,-16643.311656,0.000000,-14879.541769,...,-11737.438759,-13005.478317,-12923.710885,7770.499262,3861.585722,-5426.366784,-19013.104649,11936.136678,-5944.423324,-7017.611521
Taiwan,7828.314414,4152.758996,23380.758357,-5662.115635,24075.003352,4421.958709,6649.415075,-1763.769887,14879.541769,0.000000,...,3142.103010,1874.063452,1955.830884,22650.041031,18741.127491,9453.174985,-4133.562880,26815.678447,8935.118445,7861.930248
